# TEST
    测试相同数据下，加入了新的算法后结果有何变化

In [426]:
import numpy as np
from numpy import diag
from itertools import chain, repeat
from ipyparallel import Client
import os
import sys
import matplotlib as plt
import networkx as nx
from tqdm import trange
from numpy.linalg import norm, inv

## 通用函数

In [427]:
max_rounds = 10000
N = 10
eps = 1e-4
net_num = 10

In [428]:
def row_stochastic(A):
    return A / A.sum(axis=1, keepdims=True)

In [429]:
def adjustNorm(N):
    A = np.array(np.random.randint(2, size=(N, N)))
    np.fill_diagonal(A, 0)
    return A

In [430]:
# 生成初始观点
def init_opinion(N):
    s = np.random.rand(N)
    np.set_printoptions(precision=3)
    return s

In [431]:
def preprocessArgs(s, max_rounds):
    N = np.size(s)
    max_rounds = int(max_rounds)  # Round 0 contains the initial opinions
    z = s.copy()
    return N, z, max_rounds

In [432]:
def is_concensus(z, eps):
    a = True
    for i in range(9):
        if z[i+1] - z[i] > eps:
            a = False
            break
    return a

## self_persistence 与 alpha_global 公用函数

In [433]:
def s_weight_matrix(A, N):
    W = np.zeros((N, N))
    alpha = np.random.rand(N)
    deg_p = A.sum(axis=1)
    deg_n = A.sum(axis=0)
    alpha = (1 + (deg_p -1)/(deg_p + deg_n) - (deg_n -1)/(deg_p + deg_n)) * alpha
    for i in range(len(alpha)):
        if alpha[i] > 1:
            alpha[i] = np.float32(1)
    for i in range(len(deg_n)):
        if deg_n[i] < 1e-4:
            alpha[i] = 1
    for i in range(N):
        W[i][i] = alpha[i]
    # 这里计算出了自我坚持度 alpha
    # 下一步应该是计算影响权重矩阵
    num = A.sum(axis=1)
    for i in range(N):
        for j in range(N):
            if i != j and A[i][j] == 1:
                W[i][j] = (1 - W[i][i]) / num[i]
    return W

## 统一网络结构及初始观点

In [434]:
A = adjustNorm(N) # 生成网络G， A是G的邻接矩阵，表示哪些节点之间存在边
s = init_opinion(N) # 初始观点向量 1 * 10

In [435]:
W_s = s_weight_matrix(A, N)

## DeGroot

In [436]:
def base_weight_matrix(A, N):
    W = np.zeros((N, N))
    np.fill_diagonal(A, 1)
    num = A.sum(axis=1)
    for i in range(N):
        for j in range(N):
            if A[i][j] == 1:
                W[i][j] = 1 / num[i]
    return W

In [437]:
def base_op_dy_modle(W, s, max_rounds, eps):
    progress = []
    dynamic_progress = []
    N, z, max_rounds = preprocessArgs(s, max_rounds)
    progress.append(z)
    opinions = np.zeros((max_rounds, N))
    opinions[0, :] = s
    
    for t in range(max_rounds):
        times = t
        z = z.dot(W.T)
        if t % 1000 == 0:
            progress.append(z)
        if is_concensus(z, eps):
            progress.append(z)
            print("The opinion has been agreed. ")
            return z, progress, times
            break
        if t == max_rounds - 1:
            print("Opinions evolved {} times".format(max_rounds))
            return z, progress, times

In [438]:
print("============DeGroot===============")
W_de = base_weight_matrix(A, N)
opinion_de, progress_de, times_de = base_op_dy_modle(W_de, s, max_rounds, eps)
print(A, "\n", W_de)
print("initial_opinion:\t", s)
print("last_opinion:\t", opinion_de)
print("progress:\n", progress_de, "\n") 
print("Times:\t", times_de)

============DeGroot===============
The opinion has been agreed. 
[[1 0 0 0 0 1 0 1 0 1]
 [1 1 1 1 0 0 1 0 0 1]
 [0 1 1 1 0 0 0 0 1 1]
 [1 0 1 1 0 1 0 0 1 1]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 1 0 0 1 1]
 [1 0 0 0 0 0 1 1 0 1]
 [0 0 1 1 1 0 1 1 0 1]
 [0 0 0 0 1 0 1 0 1 0]
 [0 1 1 1 1 0 1 1 1 1]] 
 [[0.25  0.    0.    0.    0.    0.25  0.    0.25  0.    0.25 ]
 [0.167 0.167 0.167 0.167 0.    0.    0.167 0.    0.    0.167]
 [0.    0.2   0.2   0.2   0.    0.    0.    0.    0.2   0.2  ]
 [0.167 0.    0.167 0.167 0.    0.167 0.    0.    0.167 0.167]
 [0.    0.    0.    0.5   0.5   0.    0.    0.    0.    0.   ]
 [0.    0.    0.2   0.    0.2   0.2   0.    0.    0.2   0.2  ]
 [0.25  0.    0.    0.    0.    0.    0.25  0.25  0.    0.25 ]
 [0.    0.    0.167 0.167 0.167 0.    0.167 0.167 0.    0.167]
 [0.    0.    0.    0.    0.333 0.    0.333 0.    0.333 0.   ]
 [0.    0.125 0.125 0.125 0.125 0.    0.125 0.125 0.125 0.125]]
initial_opinion:	 [0.641 0.201 0.774 0.47  0.169 0.538 0.819 0.761 0.66

## alpha + global

In [439]:
def global_influence(opinion):
    g_sum = 0
    for i in range(len(opinion)):
        g_sum = g_sum + opinion[i]
    average = g_sum / (len(opinion) + 1)
    return average

In [440]:
def a_g_op_dy_modle(W, s, max_rounds, eps):
    mu_n = 0.8
    mu_g = 0.2
    is_agreed = True
    progress = []
    dynamic_progress = []
    N, z, max_rounds = preprocessArgs(s, max_rounds)
    progress.append(z)
    opinions = np.zeros((max_rounds, N))
    opinions[0, :] = s 
    for t in range(max_rounds):
        # 观点开始演化
        if t%100 == 0:        
            ave_op = global_influence(z)
            z = mu_n * z.dot(W.T) + mu_g * ave_op
        else:
            z = z.dot(W.T)
        if t % 1000 == 0:
            progress.append(z)
        if is_concensus(z, eps):
            progress.append(z)
            print("The opinion has been agreed. ")
            return z, progress, is_agreed, t
            break
        if t == max_rounds - 1:
            is_agreed = False
            print("Opinions evolved {} times\n".format(max_rounds))
            return z, progress, is_agreed, t

In [441]:
print("============alpha&global_100:1===============")
opinion_a_g, progress_a_g, is_agreed_a_g, times_a_100 = a_g_op_dy_modle(W_s, s, max_rounds, eps)
print(A, "\n", W_s)
print("initial_opinion:\t", s)
print("last_opinion:\t", opinion_a_g)
if is_agreed_a_g:
    print("progress:\n", progress_a_g, "\n")
print("Times:\t", times_a_100)

============alpha&global_100:1===============
The opinion has been agreed. 
[[1 0 0 0 0 1 0 1 0 1]
 [1 1 1 1 0 0 1 0 0 1]
 [0 1 1 1 0 0 0 0 1 1]
 [1 0 1 1 0 1 0 0 1 1]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 1 0 0 1 1]
 [1 0 0 0 0 0 1 1 0 1]
 [0 0 1 1 1 0 1 1 0 1]
 [0 0 0 0 1 0 1 0 1 0]
 [0 1 1 1 1 0 1 1 1 1]] 
 [[0.821 0.    0.    0.    0.    0.06  0.    0.06  0.    0.06 ]
 [0.19  0.051 0.19  0.19  0.    0.    0.19  0.    0.    0.19 ]
 [0.    0.061 0.755 0.061 0.    0.    0.    0.    0.061 0.061]
 [0.079 0.    0.079 0.604 0.    0.079 0.    0.    0.079 0.079]
 [0.    0.    0.    0.779 0.221 0.    0.    0.    0.    0.   ]
 [0.    0.    0.221 0.    0.221 0.115 0.    0.    0.221 0.221]
 [0.067 0.    0.    0.    0.    0.    0.8   0.067 0.    0.067]
 [0.    0.    0.128 0.128 0.128 0.    0.128 0.361 0.    0.128]
 [0.    0.    0.    0.    0.183 0.    0.183 0.    0.634 0.   ]
 [0.    0.127 0.127 0.127 0.127 0.    0.127 0.127 0.127 0.109]]
initial_opinion:	 [0.641 0.201 0.774 0.47  0.169 0.538 0.819

In [442]:
def a_g_op_dy_modle_1(W, s, max_rounds, eps):
    mu_n = 0.8
    mu_g = 0.2
    is_agreed = True
    progress = []
    dynamic_progress = []
    N, z, max_rounds = preprocessArgs(s, max_rounds)
    progress.append(z)
    opinions = np.zeros((max_rounds, N))
    opinions[0, :] = s 
    for t in range(max_rounds):
        # 观点开始演化
        ave_op = global_influence(z)
        z = mu_n * z.dot(W.T) + mu_g * ave_op
        if t % 1000 == 0:
            progress.append(z)
        if is_concensus(z, eps):
            progress.append(z)
            print("The opinion has been agreed. ")
            return z, progress, is_agreed, t
            break
        if t == max_rounds - 1:
            is_agreed = False
            print("Opinions evolved {} times\n".format(max_rounds))
            return z, progress, is_agreed, t
        
print("============alpha&global_1:1===============")
opinion_a_g_1, progress_a_g_1, is_agreed_a_g_1, times_a_1 = a_g_op_dy_modle_1(W_s, s, max_rounds, eps)
print(A, "\n", W_s)
print("initial_opinion:\t", s)
print("last_opinion:\t", opinion_a_g_1)
if is_agreed_a_g_1:
    print("progress:\n", progress_a_g_1, "\n")
print("Times:\t", times_a_1)

============alpha&global_1:1===============
The opinion has been agreed. 
[[1 0 0 0 0 1 0 1 0 1]
 [1 1 1 1 0 0 1 0 0 1]
 [0 1 1 1 0 0 0 0 1 1]
 [1 0 1 1 0 1 0 0 1 1]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 1 0 0 1 1]
 [1 0 0 0 0 0 1 1 0 1]
 [0 0 1 1 1 0 1 1 0 1]
 [0 0 0 0 1 0 1 0 1 0]
 [0 1 1 1 1 0 1 1 1 1]] 
 [[0.821 0.    0.    0.    0.    0.06  0.    0.06  0.    0.06 ]
 [0.19  0.051 0.19  0.19  0.    0.    0.19  0.    0.    0.19 ]
 [0.    0.061 0.755 0.061 0.    0.    0.    0.    0.061 0.061]
 [0.079 0.    0.079 0.604 0.    0.079 0.    0.    0.079 0.079]
 [0.    0.    0.    0.779 0.221 0.    0.    0.    0.    0.   ]
 [0.    0.    0.221 0.    0.221 0.115 0.    0.    0.221 0.221]
 [0.067 0.    0.    0.    0.    0.    0.8   0.067 0.    0.067]
 [0.    0.    0.128 0.128 0.128 0.    0.128 0.361 0.    0.128]
 [0.    0.    0.    0.    0.183 0.    0.183 0.    0.634 0.   ]
 [0.    0.127 0.127 0.127 0.127 0.    0.127 0.127 0.127 0.109]]
initial_opinion:	 [0.641 0.201 0.774 0.47  0.169 0.538 0.819 0

## self_perisinstance 

In [443]:
def s_op_dy_modle(W, s, max_rounds, eps):
    is_agreed = True
    progress = []
    dynamic_progress = []
    N, z, max_rounds = preprocessArgs(s, max_rounds)
    progress.append(z)
    opinions = np.zeros((max_rounds, N))
    opinions[0, :] = s
    
    for t in range(max_rounds):
        z = z.dot(W.T)
        if t % 1000 == 0:
            progress.append(z)
        if is_concensus(z, eps):
            progress.append(z)
            print("The opinion has been agreed. ")
            return z, progress, is_agreed, t
            break
        if t == max_rounds - 1:
            is_agreed = False
            print("Opinions evolved {} times\n".format(max_rounds))
            return z, progress, is_agreed, t

In [444]:
print("============self-persistence===============")
opinion_s, progress_s, is_agreed_s, times_s = s_op_dy_modle(W_s, s, max_rounds, eps)
print(A)
print(W_s)
print("initial_opinion:\t", s)
print("last_opinion:\t", opinion_s)
if is_agreed_s:
    print("progress:\n", progress_s, "\n")  
print("Times:\t", times_s)

============self-persistence===============
The opinion has been agreed. 
[[1 0 0 0 0 1 0 1 0 1]
 [1 1 1 1 0 0 1 0 0 1]
 [0 1 1 1 0 0 0 0 1 1]
 [1 0 1 1 0 1 0 0 1 1]
 [0 0 0 1 1 0 0 0 0 0]
 [0 0 1 0 1 1 0 0 1 1]
 [1 0 0 0 0 0 1 1 0 1]
 [0 0 1 1 1 0 1 1 0 1]
 [0 0 0 0 1 0 1 0 1 0]
 [0 1 1 1 1 0 1 1 1 1]]
[[0.821 0.    0.    0.    0.    0.06  0.    0.06  0.    0.06 ]
 [0.19  0.051 0.19  0.19  0.    0.    0.19  0.    0.    0.19 ]
 [0.    0.061 0.755 0.061 0.    0.    0.    0.    0.061 0.061]
 [0.079 0.    0.079 0.604 0.    0.079 0.    0.    0.079 0.079]
 [0.    0.    0.    0.779 0.221 0.    0.    0.    0.    0.   ]
 [0.    0.    0.221 0.    0.221 0.115 0.    0.    0.221 0.221]
 [0.067 0.    0.    0.    0.    0.    0.8   0.067 0.    0.067]
 [0.    0.    0.128 0.128 0.128 0.    0.128 0.361 0.    0.128]
 [0.    0.    0.    0.    0.183 0.    0.183 0.    0.634 0.   ]
 [0.    0.127 0.127 0.127 0.127 0.    0.127 0.127 0.127 0.109]]
initial_opinion:	 [0.641 0.201 0.774 0.47  0.169 0.538 0.819 0.7